# Import

In [21]:
import numpy as np
import pandas as pd
from numba import jit, float64, boolean, int64
import math
import matplotlib.pyplot as plt

In [22]:
import numpy as np
import pandas as pd
from numba import jit, float64, boolean
import math
import matplotlib.pyplot as plt

import sys, warnings
sys.path.append('/root/notebook/Workspace/Options/pyoption')
sys.path.append('/root/notebook/Workspace/userlibs')
import numba_recursive as nbr
import option_helper as opth

# EWM

In [23]:
x1 = np.random.random(10)
x2 = np.random.random(10)
x3 = np.random.random(10)
xs = np.hstack([x1, x2, x3])
xs

array([0.13163933, 0.44112621, 0.71167469, 0.01047671, 0.98224655,
       0.02240815, 0.94849791, 0.55360206, 0.26094773, 0.21438584,
       0.15626231, 0.23965329, 0.66117578, 0.10904576, 0.28468653,
       0.27455854, 0.15110472, 0.14325147, 0.74683435, 0.50855314,
       0.38314029, 0.26510729, 0.09659881, 0.01855824, 0.34891597,
       0.15884723, 0.63274378, 0.33487056, 0.44359786, 0.76483305])

### EWMA

In [24]:
ews_pd = pd.Series(xs).ewm(alpha=0.05, adjust=True, ignore_na=False, min_periods=1).mean().values

In [25]:
ews_true, last_state = nbr.numba_ewma(xs, alpha=0.05, state=None, adjust=True, ignore_na=False, minp=1)
print('last_state:\n', last_state)
print('ews_true:\n', ews_true)

last_state:
 [ 0.36851781 15.70722472]
ews_true:
 [0.13163933 0.29035055 0.43805402 0.32280019 0.46855412 0.38434644
 0.47785343 0.48910614 0.45825313 0.42786567 0.39637177 0.3793238
 0.40828172 0.379078   0.37028445 0.36173556 0.34363638 0.32701481
 0.36072733 0.37224896 0.37307477 0.36509451 0.34571255 0.32260879
 0.32442908 0.31318764 0.33450116 0.33452539 0.34157083 0.36851781]


In [26]:
ew1, st1 = nbr.numba_ewma(x1, alpha=0.05, state=None, adjust=True, ignore_na=False, minp=1)
# print('ew1:\t', ew1)
print('state1:\t', st1)

ew2, st2 = nbr.numba_ewma(x2, alpha=0.05, state=st1, adjust=True, ignore_na=False, minp=1)
# print('ew2:\t', ew2)
print('state2:\t', st2)

ew3, st3 = nbr.numba_ewma(x3, alpha=0.05, state=st2, adjust=True, ignore_na=False, minp=1)
# print('ew2:\t', ew3)
print('state2:\t', st3)

ews_new = np.hstack([ew1, ew2, ew3])
print(ews_new)

state1:	 [0.42786567 8.02526122]
state2:	 [ 0.37224896 12.83028155]
state2:	 [ 0.36851781 15.70722472]
[0.13163933 0.29035055 0.43805402 0.32280019 0.46855412 0.38434644
 0.47785343 0.48910614 0.45825313 0.42786567 0.39637177 0.3793238
 0.40828172 0.379078   0.37028445 0.36173556 0.34363638 0.32701481
 0.36072733 0.37224896 0.37307477 0.36509451 0.34571255 0.32260879
 0.32442908 0.31318764 0.33450116 0.33452539 0.34157083 0.36851781]


In [27]:
np.nansum((ews_pd - ews_true) ** 2)

0.0

In [28]:
np.nansum((ews_pd - ews_new) ** 2)

0.0

### EWVAR

In [29]:
ews_pd = pd.Series(xs).ewm(alpha=0.05, adjust=True, ignore_na=False, min_periods=1).var(bias=False).values

In [30]:
ews_true, last_state = nbr.numba_ewvar(xs, alpha=0.05, state=None, adjust=True, ignore_na=False, minp=1, bias=False)
print('last_state:\n', last_state)
print('ews_true:\n', ews_true)

last_state:
 [ 0.36851781  0.36851781  0.06323935 15.70722472  9.7838995  15.70722472]
ews_true:
 [       nan 0.04789106 0.0840068  0.10260449 0.16866134 0.16801271
 0.18773451 0.15744895 0.14092279 0.12909156 0.12134592 0.10988013
 0.10588508 0.10352661 0.09423229 0.08625146 0.08226564 0.07844413
 0.08573667 0.08051242 0.074246   0.06945658 0.06939668 0.07175666
 0.06673587 0.06393302 0.06623317 0.0618156  0.05850963 0.06585075]


In [31]:
ew1, st1 = nbr.numba_ewvar(x1, 0.05, adjust=True, state=None, ignore_na=True, minp=1, bias=False)
# print('ew1:\t', ew1)
print('state1:\t', st1)

ew2, st2 = nbr.numba_ewvar(x2, 0.05, adjust=True, state=st1, ignore_na=True, minp=1, bias=False)
# print('ew2:\t', ew2)
print('state2:\t', st2)

ew3, st3 = nbr.numba_ewvar(x3, 0.05, adjust=True, state=st2, ignore_na=True, minp=1, bias=False)
# print('ew2:\t', ew3)
print('state2:\t', st3)

ews_new = np.hstack([ew1, ew2, ew3])
print(ews_new)

state1:	 [0.42786567 0.42786567 0.1159035  8.02526122 6.57963157 8.02526122]
state2:	 [ 0.37224896  0.37224896  0.07614075 12.83028155  8.93833686 12.83028155]
state2:	 [ 0.36851781  0.36851781  0.06323935 15.70722472  9.7838995  15.70722472]
[       nan 0.04789106 0.0840068  0.10260449 0.16866134 0.16801271
 0.18773451 0.15744895 0.14092279 0.12909156 0.12134592 0.10988013
 0.10588508 0.10352661 0.09423229 0.08625146 0.08226564 0.07844413
 0.08573667 0.08051242 0.074246   0.06945658 0.06939668 0.07175666
 0.06673587 0.06393302 0.06623317 0.0618156  0.05850963 0.06585075]


In [32]:
np.nansum((ews_pd - ews_true) ** 2)

0.0

In [33]:
np.nansum((ews_pd - ews_new) ** 2)

0.0

### EWSTD

In [34]:
ews_pd = pd.Series(xs).ewm(alpha=0.05, adjust=True, ignore_na=False, min_periods=1).std(bias=False).values

In [35]:
ews_true, last_state = nbr.numba_ewstd(xs, alpha=0.05, state=None, adjust=True, ignore_na=False, minp=1, bias=False)
print('last_state:\n', last_state)
print('ews_true:\n', ews_true)

last_state:
 [ 0.36851781  0.36851781  0.06323935 15.70722472  9.7838995  15.70722472]
ews_true:
 [       nan 0.21884027 0.28983927 0.32031936 0.41068399 0.40989354
 0.43328341 0.39679838 0.37539684 0.35929315 0.34834741 0.33148171
 0.32539987 0.32175552 0.30697278 0.293686   0.28681987 0.28007878
 0.29280824 0.28374711 0.2724812  0.26354617 0.2634325  0.26787433
 0.25833286 0.25284979 0.25735805 0.24862744 0.24188765 0.25661401]


In [36]:
ew1, st1 = nbr.numba_ewstd(x1, 0.05, adjust=True, state=None, ignore_na=True, minp=1, bias=False)
# print('ew1:\t', ew1)
print('state1:\t', st1)

ew2, st2 = nbr.numba_ewstd(x2, 0.05, adjust=True, state=st1, ignore_na=True, minp=1, bias=False)
# print('ew2:\t', ew2)
print('state2:\t', st2)

ew3, st3 = nbr.numba_ewstd(x3, 0.05, adjust=True, state=st2, ignore_na=True, minp=1, bias=False)
# print('ew2:\t', ew3)
print('state2:\t', st3)

ews_new = np.hstack([ew1, ew2, ew3])
print(ews_new)

state1:	 [0.42786567 0.42786567 0.1159035  8.02526122 6.57963157 8.02526122]
state2:	 [ 0.37224896  0.37224896  0.07614075 12.83028155  8.93833686 12.83028155]
state2:	 [ 0.36851781  0.36851781  0.06323935 15.70722472  9.7838995  15.70722472]
[       nan 0.21884027 0.28983927 0.32031936 0.41068399 0.40989354
 0.43328341 0.39679838 0.37539684 0.35929315 0.34834741 0.33148171
 0.32539987 0.32175552 0.30697278 0.293686   0.28681987 0.28007878
 0.29280824 0.28374711 0.2724812  0.26354617 0.2634325  0.26787433
 0.25833286 0.25284979 0.25735805 0.24862744 0.24188765 0.25661401]


In [37]:
np.nansum((ews_pd - ews_true) ** 2)

0.0

In [38]:
np.nansum((ews_pd - ews_new) ** 2)

0.0

### OLS

#### Normal

In [45]:
x = np.random.random(300)
y = 2 * x + np.random.random(300)

In [46]:
nbr.numba_ols_beta(x, y)

2.77928817312982

In [47]:
from sklearn.linear_model import LinearRegression
slope = LinearRegression(fit_intercept=False).fit(x[:, None], y).coef_
slope

array([2.77928817])

In [48]:
slope, _, _, _ = np.linalg.lstsq(x[:, None], y, rcond=None)
slope

array([2.77928817])

#### Rolling by rows

In [70]:
x = np.random.random(300)
y = 2 * x + np.random.random(300)
rs = nbr.numba_rollreg(x, y, 30, 10)
rs[-10:]

array([2.80511995, 2.83401541, 2.79615349, 2.81231063, 2.80882536,
       2.84283015, 2.8582495 , 2.86902289, 2.88094698, 2.88932388])

#### Rolling by time frequency

In [77]:
x = np.random.random(300)
y = 2 * x + np.random.random(300)
ts = pd.date_range('20180101', periods=300, freq='2d').values
tfreq = np.timedelta64(30, 'D')
rs = nbr.numba_rollreg_ts(x, y, ts, tfreq, 10)
rs[-10:]

array([2.57196712, 2.69718741, 2.79209265, 2.87718925, 2.92561233,
       2.71240482, 2.69640079, 2.77442757, 2.73339031, 2.74978372])